In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON

In [2]:
method_algo = 'mfcc'
model_algo = 'rnn'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [3]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # Simple RNN
        keras.layers.SimpleRNN(512, input_shape=[inputs.shape[1], inputs.shape[2]]),

#         keras.layers.Dropout(0.3),

#         # output layer
#         keras.layers.Dense(len(mapping), activation='softmax'),
    ])

    return model

In [4]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(inputs_train,
                                                                            targets_train,
                                                                            test_size=0.2,
                                                                            stratify=targets_train,
                                                                            random_state=random_state)


In [5]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [6]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [8]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
                    validation_data=(inputs_valid, targets_valid),
                    epochs=1000,
                    batch_size=128,
                    callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/1000
89/89 [==============================] - 9s 89ms/step - loss: 13.5674 - accuracy: 0.0086 - val_loss: 13.3025 - val_accuracy: 0.0096
Epoch 2/1000
89/89 [==============================] - 7s 79ms/step - loss: 13.3727 - accuracy: 0.0112 - val_loss: 12.8988 - val_accuracy: 0.0127
Epoch 3/1000
89/89 [==============================] - 7s 79ms/step - loss: 12.8793 - accuracy: 0.0118 - val_loss: 12.8469 - val_accuracy: 0.0127
Epoch 4/1000
89/89 [==============================] - 7s 78ms/step - loss: 12.8251 - accuracy: 0.0125 - val_loss: 12.7548 - val_accuracy: 0.0120
Epoch 5/1000
89/89 [==============================] - 7s 79ms/step - loss: 12.6052 - accuracy: 0.0124 - val_loss: 12.7354 - val_accuracy: 0.0149
Epoch 6/1000
89/89 [==============================] - 7s 78ms/step - loss: 12.6336 - accuracy: 0.0150 - val_loss: 12.7588 - val_accuracy: 0.0141
Epoch 7/1000
89/89 [==============================] - 7s 79ms/step - loss: 12.7708 - accuracy: 0.0139 - val_loss: 12.8397 - val_ac

89/89 [==============================] - 7s 78ms/step - loss: 8.5205 - accuracy: 0.0160 - val_loss: 8.9078 - val_accuracy: 0.0134
Epoch 58/1000
89/89 [==============================] - 7s 79ms/step - loss: 8.5618 - accuracy: 0.0130 - val_loss: 8.8868 - val_accuracy: 0.0134
Epoch 59/1000
89/89 [==============================] - 7s 79ms/step - loss: 8.5819 - accuracy: 0.0124 - val_loss: 8.8887 - val_accuracy: 0.0131
Epoch 60/1000
89/89 [==============================] - 7s 78ms/step - loss: 8.4608 - accuracy: 0.0151 - val_loss: 8.8930 - val_accuracy: 0.0127
Epoch 61/1000
89/89 [==============================] - 7s 78ms/step - loss: 8.4840 - accuracy: 0.0157 - val_loss: 8.8362 - val_accuracy: 0.0127
Epoch 62/1000
89/89 [==============================] - 7s 79ms/step - loss: 8.4221 - accuracy: 0.0173 - val_loss: 8.8144 - val_accuracy: 0.0120
Epoch 63/1000
89/89 [==============================] - 7s 78ms/step - loss: 8.3876 - accuracy: 0.0162 - val_loss: 8.7664 - val_accuracy: 0.0113
Epoch 

Epoch 114/1000
89/89 [==============================] - 7s 79ms/step - loss: 7.7599 - accuracy: 0.0156 - val_loss: 8.1549 - val_accuracy: 0.0067
Epoch 115/1000
89/89 [==============================] - 7s 78ms/step - loss: 7.7096 - accuracy: 0.0146 - val_loss: 8.1421 - val_accuracy: 0.0067
Epoch 116/1000
89/89 [==============================] - 7s 79ms/step - loss: 7.6888 - accuracy: 0.0163 - val_loss: 8.1366 - val_accuracy: 0.0071
Epoch 117/1000
89/89 [==============================] - 7s 78ms/step - loss: 7.6589 - accuracy: 0.0139 - val_loss: 8.1187 - val_accuracy: 0.0067
Epoch 118/1000
89/89 [==============================] - 7s 79ms/step - loss: 7.6903 - accuracy: 0.0123 - val_loss: 8.1296 - val_accuracy: 0.0067
Epoch 119/1000
89/89 [==============================] - 7s 78ms/step - loss: 7.7347 - accuracy: 0.0131 - val_loss: 8.1042 - val_accuracy: 0.0074
Epoch 120/1000
89/89 [==============================] - 7s 78ms/step - loss: 7.6468 - accuracy: 0.0145 - val_loss: 8.1670 - val_ac

89/89 [==============================] - 13s 148ms/step - loss: 7.3880 - accuracy: 0.0139 - val_loss: 7.8755 - val_accuracy: 0.0096
Epoch 171/1000
89/89 [==============================] - 13s 148ms/step - loss: 7.3648 - accuracy: 0.0139 - val_loss: 7.8858 - val_accuracy: 0.0099
Epoch 172/1000
89/89 [==============================] - 13s 147ms/step - loss: 7.3765 - accuracy: 0.0125 - val_loss: 7.9083 - val_accuracy: 0.0092
Epoch 173/1000
89/89 [==============================] - 13s 147ms/step - loss: 7.3666 - accuracy: 0.0148 - val_loss: 7.9069 - val_accuracy: 0.0085
Epoch 174/1000
89/89 [==============================] - 13s 149ms/step - loss: 7.3598 - accuracy: 0.0136 - val_loss: 7.9132 - val_accuracy: 0.0085
Epoch 175/1000
89/89 [==============================] - 12s 140ms/step - loss: 7.3740 - accuracy: 0.0140 - val_loss: 7.9258 - val_accuracy: 0.0088
Epoch 176/1000
89/89 [==============================] - 13s 146ms/step - loss: 7.3286 - accuracy: 0.0137 - val_loss: 7.8959 - val_acc

Epoch 226/1000
89/89 [==============================] - 13s 145ms/step - loss: 7.1760 - accuracy: 0.0146 - val_loss: 7.7688 - val_accuracy: 0.0088
Epoch 227/1000
89/89 [==============================] - 13s 143ms/step - loss: 7.2490 - accuracy: 0.0118 - val_loss: 7.7304 - val_accuracy: 0.0099
Epoch 228/1000
89/89 [==============================] - 13s 146ms/step - loss: 7.1960 - accuracy: 0.0115 - val_loss: 7.7300 - val_accuracy: 0.0099
Epoch 229/1000
89/89 [==============================] - 13s 149ms/step - loss: 7.2331 - accuracy: 0.0119 - val_loss: 7.7370 - val_accuracy: 0.0106
Epoch 230/1000
89/89 [==============================] - 13s 147ms/step - loss: 7.2679 - accuracy: 0.0139 - val_loss: 7.7360 - val_accuracy: 0.0106
Epoch 231/1000
89/89 [==============================] - 13s 144ms/step - loss: 7.2271 - accuracy: 0.0129 - val_loss: 7.7261 - val_accuracy: 0.0106
Epoch 232/1000
89/89 [==============================] - 13s 145ms/step - loss: 7.2118 - accuracy: 0.0124 - val_loss: 7

...

Epoch 735/1000
89/89 [==============================] - 13s 146ms/step - loss: 6.7484 - accuracy: 0.0135 - val_loss: 7.3003 - val_accuracy: 0.0092
Epoch 736/1000
89/89 [==============================] - 12s 141ms/step - loss: 6.7481 - accuracy: 0.0111 - val_loss: 7.2926 - val_accuracy: 0.0096
Epoch 737/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.7416 - accuracy: 0.0139 - val_loss: 7.2769 - val_accuracy: 0.0099
Epoch 738/1000
89/89 [==============================] - 13s 143ms/step - loss: 6.7349 - accuracy: 0.0134 - val_loss: 7.2804 - val_accuracy: 0.0099
Epoch 739/1000
89/89 [==============================] - 13s 146ms/step - loss: 6.7416 - accuracy: 0.0130 - val_loss: 7.2873 - val_accuracy: 0.0099
Epoch 740/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.7344 - accuracy: 0.0124 - val_loss: 7.2852 - val_accuracy: 0.0099
Epoch 741/1000
89/89 [==============================] - 13s 148ms/step - loss: 6.7562 - accuracy: 0.0130 - val_loss: 7

Epoch 791/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7170 - accuracy: 0.0138 - val_loss: 7.2401 - val_accuracy: 0.0092
Epoch 792/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7364 - accuracy: 0.0125 - val_loss: 7.2425 - val_accuracy: 0.0092
Epoch 793/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7327 - accuracy: 0.0150 - val_loss: 7.2413 - val_accuracy: 0.0096
Epoch 794/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7402 - accuracy: 0.0137 - val_loss: 7.2367 - val_accuracy: 0.0092
Epoch 795/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7530 - accuracy: 0.0127 - val_loss: 7.2342 - val_accuracy: 0.0092
Epoch 796/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.6795 - accuracy: 0.0140 - val_loss: 7.2334 - val_accuracy: 0.0092
Epoch 797/1000
89/89 [==============================] - 13s 146ms/step - loss: 6.7329 - accuracy: 0.0141 - val_loss: 7

Epoch 847/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7440 - accuracy: 0.0152 - val_loss: 7.2939 - val_accuracy: 0.0085
Epoch 848/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.7348 - accuracy: 0.0127 - val_loss: 7.2872 - val_accuracy: 0.0088
Epoch 849/1000
89/89 [==============================] - 13s 148ms/step - loss: 6.7498 - accuracy: 0.0122 - val_loss: 7.2890 - val_accuracy: 0.0088
Epoch 850/1000
89/89 [==============================] - 13s 146ms/step - loss: 6.7382 - accuracy: 0.0144 - val_loss: 7.2860 - val_accuracy: 0.0085
Epoch 851/1000
89/89 [==============================] - 13s 146ms/step - loss: 6.7631 - accuracy: 0.0116 - val_loss: 7.2727 - val_accuracy: 0.0085
Epoch 852/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.8017 - accuracy: 0.0130 - val_loss: 7.2843 - val_accuracy: 0.0088
Epoch 853/1000
89/89 [==============================] - 13s 142ms/step - loss: 6.7495 - accuracy: 0.0136 - val_loss: 7

Epoch 903/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.6878 - accuracy: 0.0145 - val_loss: 7.2714 - val_accuracy: 0.0085
Epoch 904/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.7048 - accuracy: 0.0134 - val_loss: 7.2926 - val_accuracy: 0.0085
Epoch 905/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.6881 - accuracy: 0.0137 - val_loss: 7.2852 - val_accuracy: 0.0085
Epoch 906/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7472 - accuracy: 0.0141 - val_loss: 7.2669 - val_accuracy: 0.0088
Epoch 907/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7489 - accuracy: 0.0154 - val_loss: 7.2758 - val_accuracy: 0.0088
Epoch 908/1000
89/89 [==============================] - 13s 144ms/step - loss: 6.7531 - accuracy: 0.0142 - val_loss: 7.2746 - val_accuracy: 0.0088
Epoch 909/1000
89/89 [==============================] - 13s 146ms/step - loss: 6.7545 - accuracy: 0.0148 - val_loss: 7

Epoch 959/1000
89/89 [==============================] - 13s 149ms/step - loss: 6.7045 - accuracy: 0.0133 - val_loss: 7.2561 - val_accuracy: 0.0092
Epoch 960/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.6663 - accuracy: 0.0127 - val_loss: 7.2295 - val_accuracy: 0.0088
Epoch 961/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.6803 - accuracy: 0.0145 - val_loss: 7.2160 - val_accuracy: 0.0088
Epoch 962/1000
89/89 [==============================] - 13s 146ms/step - loss: 6.6480 - accuracy: 0.0144 - val_loss: 7.2097 - val_accuracy: 0.0092
Epoch 963/1000
89/89 [==============================] - 13s 147ms/step - loss: 6.7040 - accuracy: 0.0116 - val_loss: 7.1983 - val_accuracy: 0.0088
Epoch 964/1000
89/89 [==============================] - 13s 145ms/step - loss: 6.7050 - accuracy: 0.0122 - val_loss: 7.1930 - val_accuracy: 0.0088
Epoch 965/1000
89/89 [==============================] - 13s 148ms/step - loss: 6.6759 - accuracy: 0.0126 - val_loss: 7

In [9]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [10]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

28/28 [==============================] - 1s 49ms/step - loss: 7.2651 - accuracy: 0.0088


In [11]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                           f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')